In [1]:
!pip install Sastrawi
!pip install emoji

In [2]:
import pandas as pd

# Misalkan scrapreview sudah diubah ke DataFrame
app_reviews_df = pd.read_csv('/content/dataset_scaraping_5000.csv')
# Fungsi untuk memberi label berdasarkan skor
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

# Tambahkan kolom 'sentiment' ke DataFrame
app_reviews_df['sentiment'] = app_reviews_df['score'].apply(label_sentiment)

# Simpan ke CSV untuk memastikan hasilnya
app_reviews_df.to_csv('ulasan_dengan_label.csv', index=False)

# Cek jumlah data dan distribusi sentimen
print(f"Jumlah ulasan: {len(app_reviews_df)}")
print(app_reviews_df['sentiment'].value_counts())

Jumlah ulasan: 5000
sentiment
negatif    3168
positif    1457
netral      375
Name: count, dtype: int64


In [3]:
import pandas as pd
import re
import string
import requests
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Misalkan data sudah di-load dari CSV hasil langkah sebelumnya
app_reviews_df = pd.read_csv('ulasan_dengan_label.csv')

def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = remove_emoji(text)  # Hapus emoji


    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

# Fungsi stemming
stemmer = StemmerFactory().create_stemmer()
def stemmingText(text):
    words = text.split()
    return ' '.join([stemmer.stem(word) for word in words])

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def remove_stopwords(word_list):
    stopwords_indonesia = stopwords.words('indonesian')
    no_words = [w for w in word_list if w.lower() not in stopwords_indonesia]
    return no_words

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

def load_slangwords():
    slangwords = {}
    url = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Memastikan permintaan berhasil
        # File berformat seperti dictionary Python, jadi kita evaluasi isinya
        slangwords = eval(response.text)  # Konversi string ke dictionary
        return slangwords
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil slangword: {e}")
        return {}

slangwords = load_slangwords()

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

def preprocess(text):
    text = cleaningText(text)                # 1. Cleaning
    text = fix_slangwords(text)              # 2. Fix slangwords
    text = casefoldingText(text)             # 3. Case folding
    text = stemmingText(text)                # 4. Stemming
    tokens = tokenizingText(text)            # 5. Tokenizing
    tokens = remove_stopwords(tokens)        # 6. Stopword Removal
    fixed_text = ' '.join(tokens)            # 7. Gabung token jadi kalimat
    return fixed_text


# Terapkan preprocessing
app_reviews_df['clean_text'] = app_reviews_df['content'].apply(preprocess)

# Hapus baris dengan teks kosong
app_reviews_df = app_reviews_df[app_reviews_df['clean_text'] != '']

app_reviews_df = app_reviews_df.dropna()

# Hapus duplikat
app_reviews_df = app_reviews_df.drop_duplicates(subset=['clean_text'])

# Simpan hasil
app_reviews_df.to_csv('ulasan_clean.csv', index=False)

# Cek beberapa baris
print(app_reviews_df[['clean_text', 'sentiment']].head())
print(f"Jumlah data setelah preprocessing: {len(app_reviews_df)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                          clean_text sentiment
0  aplikasi nambah dosa sya pakai kali apk bikin ...   negatif
3  kecewa parahhh sengaja kasih finger print upgr...   negatif
4  aplikasi membingunkan fiturnya scrol pisah men...   negatif
5  mohon perihal top up uang elektronik jam malam...    netral
6  kecewa top up gak masuk masuk nunggu harikalo ...   negatif
Jumlah data setelah preprocessing: 3468


In [4]:
clean_df = pd.read_csv('ulasan_clean.csv')

In [5]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              3468 non-null   object
 1   userName              3468 non-null   object
 2   userImage             3468 non-null   object
 3   content               3468 non-null   object
 4   score                 3468 non-null   int64 
 5   thumbsUpCount         3468 non-null   int64 
 6   reviewCreatedVersion  3468 non-null   object
 7   at                    3468 non-null   object
 8   replyContent          3468 non-null   object
 9   repliedAt             3468 non-null   object
 10  appVersion            3468 non-null   object
 11  sentiment             3468 non-null   object
 12  clean_text            3468 non-null   object
dtypes: int64(2), object(11)
memory usage: 352.3+ KB


# SVM TF-IDF

In [6]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Load data yang sudah dibersihkan
clean_df = pd.read_csv('ulasan_clean.csv')

# Mapping label ke angka
label_map = {'negatif': 0, 'netral': 1, 'positif': 2}
clean_df['label'] = clean_df['sentiment'].map(label_map)

# Ekstraksi fitur TF-IDF
tfidf = TfidfVectorizer(max_features=3468)  # ambil fitur top
X = tfidf.fit_transform(clean_df['clean_text']).toarray()
y = clean_df['label']

# Split data 80% train / 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Training SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Prediksi
y_pred_train_svm = svm_model.predict(X_train)
y_pred_test_svm = svm_model.predict(X_test)

# Evaluasi akurasi
accuracy_train_svm = accuracy_score(y_train, y_pred_train_svm)
accuracy_test_svm = accuracy_score(y_test, y_pred_test_svm)

# Tampilkan hasil
print("SVM - Accuracy Train:", accuracy_train_svm)
print("SVM - Accuracy Test:", accuracy_test_svm)

SVM - Accuracy Train: 0.8767123287671232
SVM - Accuracy Test: 0.8746397694524496


In [7]:
import random
# Menggunakan model untuk prediksi
y_pred_test_svm = svm_model.predict(X_test)  # Prediksi untuk data uji

# Menampilkan hasil prediksi secara acak
random_indices = random.sample(range(len(X_test)), 10)  # Pilih 10 indeks acak

print("Prediksi Sentimen untuk Data Uji (Acak):")
for i in random_indices:
    print(f"Text: {clean_df['clean_text'].iloc[i]}, Prediksi: {y_pred_test_svm[i]}")

Prediksi Sentimen untuk Data Uji (Acak):
Text: kecewa layan ikan gopay tranfer bank proses saldo kosong tunggu terang proses lapor cs alas suruh nunggu kerja kecewa aplikasi, Prediksi: 0
Text: tolong tunggu aktivasi kasih tulis menit ditungguin jam kabar tolong tulis gak biar gak nunggu loh tulis, Prediksi: 0
Text: pakai gopay udh bertaunbaru kecewa kali upgrade gopay tabung top up saldo masuk uang hilang masuk bantu ga upload foto buktiauto pindah belah mah, Prediksi: 0
Text: kondisi urgent error gak transfer nerima transfer cepat banget apk aneh parah, Prediksi: 0
Text: tampil ampas banget bingungin tolong gausah designnya fungsional, Prediksi: 0
Text: gopay parah ga rekomended akses susah bayar tagih susah saldo situ kemarin komplin susah akses nyata tetep solusi testimoni negatif gitu masak ga on the way jalan uninstall nihsalam, Prediksi: 0
Text: aplikasi parahh seduniaudah verfikasi wajah pencahyaan tetep gagalmana uang aplikasi butuh bangetkirim bantu gak solusiparahhh, Prediksi

# RF-WordVec

In [11]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Load data bersih
data = pd.read_csv('ulasan_clean.csv')
label_map = {'negatif': 0, 'netral': 1, 'positif': 2}
data['label'] = data['sentiment'].map(label_map)

# Tokenizing untuk Word2Vec
data['tokens'] = data['clean_text'].apply(nltk.word_tokenize)

# Train Word2Vec model
w2v_model = Word2Vec(sentences=data['tokens'], vector_size=100, window=5, min_count=1, sg=1)

# Fungsi untuk merata-rata embedding setiap kalimat
def vector_average(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

# Konversi semua text jadi vektor
X_vectors = np.array([vector_average(tokens) for tokens in data['tokens']])
y = data['label']

# Split data
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42, stratify=y
)

# Training Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_w2v, y_train_w2v)

y_pred_train_rf = rf_model.predict(X_train_w2v)
y_pred_test_rf = rf_model.predict(X_test_w2v)

print("Random Forest - accuracy_train:", accuracy_score(y_pred_train_rf, y_train_w2v))
print("Random Forest - accuracy_test :", accuracy_score(y_pred_test_rf, y_test_w2v))

Random Forest - accuracy_train: 1.0
Random Forest - accuracy_test : 0.8746397694524496


In [12]:
print(clean_df['sentiment'].value_counts())
print(f"Training data: {len(X_train_w2v)}")
print(f"Testing data: {len(X_test_w2v)}")

sentiment
negatif    3033
netral      357
positif      78
Name: count, dtype: int64
Training data: 2774
Testing data: 694


In [14]:
import random
from imblearn.over_sampling import SMOTE
from collections import Counter


smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_w2v, y_pred_train_rf)

print(f"Data training balanced: {Counter(y_train_bal)}")

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x79a0d565aac0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error
Exception ignored on calling ctypes callback function: <functio

Data training balanced: Counter({0: 2426, 1: 2426, 2: 2426})


In [15]:
# Melatih model Random Forest dengan data yang seimbang
rf_model_bal = RandomForestClassifier(random_state=42)
rf_model_bal.fit(X_train_bal, y_train_bal)

# Prediksi pada data uji
y_pred_bal = rf_model_bal.predict(X_test_w2v)

# Evaluasi Model
print("Random Forest (Data Seimbang) - Classification Report:")
print(classification_report(y_pred_test_rf, y_pred_bal))
print("Akurasi (Test Data):", accuracy_score(y_pred_test_rf, y_pred_bal))


Random Forest (Data Seimbang) - Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       692
           1       0.00      0.00      0.00         1
           2       0.07      1.00      0.12         1

    accuracy                           0.92       694
   macro avg       0.36      0.64      0.36       694
weighted avg       1.00      0.92      0.95       694

Akurasi (Test Data): 0.9164265129682997


In [16]:
# Menggunakan model untuk prediksi
y_pred_bal = rf_model_bal.predict(X_test_w2v)  # Prediksi untuk data uji

# Menampilkan hasil prediksi secara acak
random_indices = random.sample(range(len(X_test_w2v)), 10)  # Pilih 10 indeks acak

print("Prediksi Sentimen untuk Data Uji (Acak):")
for i in random_indices:
    print(f"Text: {clean_df['clean_text'].iloc[i]}, Prediksi: {y_pred_bal[i]}")

Prediksi Sentimen untuk Data Uji (Acak):
Text: kecewa aplikasi pakai aplikasi transfer status memprosesdi suruh hubung cs jawabanudh proses gak uang nya pakai ngopi kali iya, Prediksi: 0
Text: ken beli makan pakai qris sulit gmn uang tahan gopay transfer notif transaksi curiga transaksi bayar makan tolong uang gk pakai, Prediksi: 0
Text: aplikasi buruk sekalibanyak no wa masuk ancamanan sebar data sesuai iklan, Prediksi: 0
Text: top up gopay ga masuk kadang masuk lambat jm masuk kecewa banget skarang top up saldo masuk skali, Prediksi: 0
Text: download pakai aplikasi gopay iniuntuk alam kali uang tf hilang bank terima masukmau hubung gopay susah alam buruk apk gopaymending pakai apk dana sajalancar apk dana ukses, Prediksi: 0
Text: x transaksi top up game berturutturut voucher cashback pakai payment ga masuk cashback nya lumayan top up gopay gada kendala lapor responnya g niat bikin promo ga ush nawarin spam notifikasi bikin tarik bodoh males pakai gopay pindah aplikasi belah, Prediksi

# LR TF-IDF

In [19]:
from sklearn.linear_model import LogisticRegression
# Ekstraksi fitur menggunakan TF-IDF
tfidf = TfidfVectorizer(max_features=3468)  # Ambil 3000 fitur teratas
X = tfidf.fit_transform(clean_df['clean_text']).toarray()
y = clean_df['label']

# Pembagian data 80/20 (80% untuk latih, 20% untuk uji)
X_train_8020, X_test_8020, y_train_8020, y_test_8020 = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Logistic Regression
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_8020, y_train_8020)

# Prediksi dengan Logistic Regression
y_pred_lr = lr_model.predict(X_test_8020)

# Evaluasi Logistic Regression
print("Logistic Regression - Classification Report:")
print(classification_report(y_test_8020, y_pred_lr))
print("Logistic Regression - accuracy_train:", accuracy_score(y_train_8020, lr_model.predict(X_train_8020)))
print("Logistic Regression - accuracy_test:", accuracy_score(y_test_8020, y_pred_lr))


Logistic Regression - Classification Report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       604
           1       0.00      0.00      0.00        76
           2       0.00      0.00      0.00        14

    accuracy                           0.87       694
   macro avg       0.29      0.33      0.31       694
weighted avg       0.76      0.87      0.81       694

Logistic Regression - accuracy_train: 0.8795962509012256
Logistic Regression - accuracy_test: 0.8688760806916427


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
import random

# Menggunakan model untuk prediksi
y_pred_lr = lr_model.predict(X_test_8020)  # Prediksi untuk data uji

# Menampilkan hasil prediksi secara acak
random_indices = random.sample(range(len(X_test_8020)), 10)  # Pilih 10 indeks acak

print("Prediksi Sentimen untuk Data Uji (Acak):")
for i in random_indices:
    print(f"Text: {clean_df['clean_text'].iloc[i]}, Prediksi: {y_pred_lr[i]}")

Prediksi Sentimen untuk Data Uji (Acak):
Text: gopay sih suami salah nomer nomer aktif nomernya daftar gopay aneh nya transfer hasil duit lenyap bgtu, Prediksi: 0
Text: kesan maksa banget marketing ajarin banyakin doang kesan nyaman utamain gerak bidang uang elektronik tuju transaksi uang bayar transfer bayar paksa upgrade buang busukk, Prediksi: 0
Text: kecewa aplikasi pakai aplikasi transfer status memprosesdi suruh hubung cs jawabanudh proses gak uang nya pakai ngopi kali iya, Prediksi: 0
Text: bintang laku beli google play kali game mobile legends cashback parah go pay tulis cashback sd rp minimal beli rp cashback nya beli wdp loh harga rp, Prediksi: 0
Text: mudah kecoh tawar promo sistem kerja tipu promo hasil promo mengajulan banding hasil ikan aplikasi bagus pilih metode payment kecewa, Prediksi: 0
Text: kecewa top up masuk pemberitahuan data transaksi masuk saldo masuk butuh bantu live chat susah banget lambat idulfitri taruh halaman biar costumer taro halaman bantu cari susah,